# 私募基金信息数据库

## 总结


## 思路
Selenium和Requests/BeautifulSoup

## 应用
1. 私募基金求职
2. 未来客户预测：结合客户画像分析

## 步骤
1. 读取页数：从'共 23569 条记录，共236页'截取pages (236)
2. 读取每页的所有链接数
    1. 等候并点击登录确认
    1. 爬取第一页,append到`df`
    2. 点击（Click）下一页，休眠1秒
    3. 直到'paginate_button paginate_number current' == pages (236)
    4. 登录每一个链接，使用KYC中的`result_m`，merge其信息到`df`中
3. 打开每个链接爬取信息并append到`df`
4. 记录新增/删除等变动信息
5. 数据可视化分析
6. 性能优化：Big O

## 当前总数
共 23569 条记录，共236页 2018-5-11

## 问题
1. 问什么不用定义Global？

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests, time, re, math, openpyxl, datetime, os, shutil, psutil
from tqdm import *
import xlwings as xw
from selenium import webdriver
#from splinter import Browser
#from pandas.io.json import json_normalize

# Def get position function: 增加程序对数据源结构变化的稳定性
def f_p(df, feature):
    feature_list = df.iloc[:,0].get_values().tolist() # 第一列为特征名称，Index为连续自然数
    return [i for i,x in enumerate(feature_list) if x == feature][0]

# Def Check Nan Function
# https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-in-python
def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

# Get current file path for migration consistency
path = os.getcwd().replace('\\','/') #r'%s' % os.getcwd().replace('\\','/')

# 定义函数：从文本中根据数字类型提取Int或者Float数值
def find_number(text):
    if len(re.findall(r'\.',text)) == 0:
        return float(re.findall(r'\d+',text.replace(",", ""))[0])
    else:
        return float(re.findall(r'\d+\.\d+',text.replace(",", ""))[0])

url = 'http://gs.amac.org.cn/amac-infodisc/res/pof/manager/index.html'
index_m = ['机构诚信信息', '基金管理人全称(中文)', '基金管理人全称(英文)', '登记编号', '组织机构代码', '登记时间',
       '成立时间', '注册地址', '办公地址', '注册资本(万元)(人民币)', '实缴资本(万元)(人民币)', '企业性质',
       '注册资本实缴比例', '管理基金主要类别', '申请的其他业务类型', '员工人数', '机构网址', '是否为会员',
       '法律意见书状态', '律师事务所名称', '律师姓名', '法定代表人/执行事务合伙人(委派代表)姓名', '是否有从业资格',
       '资格取得方式', '法定代表人/执行事务合伙人(委派代表)工作履历', '高管情况', '暂行办法实施前成立的基金',
       '暂行办法实施后成立的基金', '机构信息最后更新时间', '特别提示信息', '查询网址/二维码', '存续产品数量',
       '累计发行产品数量']

In [8]:
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)

# 等候并点击登录确认
time.sleep(6)
driver.find_element_by_xpath("//button[@class='ui-button ui-widget ui-state-default ui-corner-all ui-button-text-only'][@type='button']").click()

# 生成列表df
df = pd.DataFrame(columns=['名称','网址'])

# 定义信息爬取函数：重复调取
def parser():
    global df
    content = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(content,'lxml')

    table = soup.find('table', attrs={'class':'table table-center dataTable no-footer'})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        col_name = row.find_all('td')[1].text.strip()
        col_link = 'http://gs.amac.org.cn/amac-infodisc/res/pof/manager/' + row.find('a')['href'].strip()
        df = df.append({'名称':col_name,'网址':col_link},ignore_index=True) # Get rid of empty values

# 下一页
def next_page():
    #global driver
    driver.find_element_by_xpath("//a[@class='paginate_button next']").click()

# 确定页数
content = driver.page_source.encode('utf-8')
soup = BeautifulSoup(content,'lxml')
funds_number = int(soup.find('div',class_='dataTables_info').get_text().split('共')[1].split('条')[0].strip())
pages_number = int(soup.find('div',class_='dataTables_info').get_text().split('共')[-1].split('页')[0].strip())

In [9]:
# Append df
for i in tqdm(range(pages_number)):
    parser()
    next_page()
    #time.sleep(0.01)

# Save df
df.to_excel('funds_list.xlsx',encoding='gb18030')

100%|██████████| 1179/1179 [03:06<00:00,  6.32it/s]


In [ ]:
df = pd.read_excel('funds_list.xlsx',encoding='gb18030')

## Append df的性能优化
V1.0 基础功能 2018-5-11

V1.1 使用Numpy Array代替Pandas DataFrame

V2.0 使用PhantomJS

In [13]:
# 生成列表df_m
df_m = pd.DataFrame(columns=index_m)

# Def result_m Function
def get_result_m(url):
###V1.0
    headers = {'User-Agent': UserAgent().random}
    r = requests.get(url,headers)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text,'lxml')
    # Any improvements possibility?
    df1 = pd.DataFrame(columns=['title'])
    df2 = pd.DataFrame(columns=['content'])
    for title in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-title'):
        df1 = df1.append({'title':title.text[:-1]},ignore_index=True)
    for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
        df2 = df2.append({'content':content.text.strip(' &nbsp\r\n')},ignore_index=True)
    result_m = pd.concat([df1,df2],axis=1)
    #result_m = result_m[result_m['content'] != '']
    result_m = result_m.reset_index(drop=True)
    result_m.loc[len(result_m),'title'] = '查询网址/二维码'
    result_m.loc[len(result_m)-1,'content'] = url_m
    result_m.content[1] = result_m.content[1].split('\n')[1] #'基金管理人全称(中文)'名称清理

    # 存续产品数量
    ## 程序改进：合并dataframe速度提升？
    num3 = 0
    for fund in soup.find_all('a',href=True,class_=False,onclick=False):
        url = str('http://gs.amac.org.cn/amac-infodisc/res/pof/fund/'+fund['href'].split('/')[2])
        headers = {'User-Agent': str(UserAgent().chrome)}
        r = requests.get(url,headers)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text,'lxml')
        df = pd.DataFrame(columns=['title'])
        df2 = pd.DataFrame(columns=['content'])
        for title in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-title'):
            df = df.append({'title':title.text[:-1]},ignore_index=True)
        for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
            df2 = df2.append({'content':content.text},ignore_index=True)
        result_m_f = pd.concat([df,df2],axis=1).loc[lambda result_m_f: result_m_f['title'] == '运作状态', :]
        if str(result_m_f.content.iloc[0]) == '正在运作':
            num3 += 1
    result_m = result_m.append({'title':'存续产品数量', 'content':num3},ignore_index=True)

    # 累计发行产品数量
    num1=num2=0
    try:
        num2 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施后成立的基金', :].iloc[0,1].split('月报'))-1
        num1 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施前成立的基金', :].iloc[0,1].split('月报'))-1
    except:
        pass
    finally:
        result_m = result_m.append({'title':'累计发行产品数量', 'content':num1+num2},ignore_index=True)
    
    row = result_m.transpose().iloc[1]
###
#1. 第二行修改url_m为url
# ###V1.1
#     headers = {'User-Agent': UserAgent().random}
#     r = requests.get(url,headers)
#     r.encoding = 'utf-8'
#     soup = BeautifulSoup(r.text,'lxml')

#     df_m_row = pd.DataFrame(columns=['content'])
#     for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
#         df_m_row = df_m_row.append({'content':content.text.strip(' &nbsp\r\n')},ignore_index=True)
#     df_m_row = df_m_row[df_m_row['content'] != '']
#     df_m_row = df_m_row.reset_index(drop=True)
#     df_m_row.loc[len(df_m_row),'content'] = url_m
#     df_m_row.content[0] = df_m_row.content[0].split('\n')[1]

#     # 存续产品数量
#     ## 程序改进：合并dataframe速度提升？
#     num3 = 0
#     for fund in soup.find_all('a',href=True,class_=False,onclick=False):
#         url = str('http://gs.amac.org.cn/amac-infodisc/res/pof/fund/'+fund['href'].split('/')[2])
#         headers = {'User-Agent': UserAgent().random}
#         r = requests.get(url,headers)
#         r.encoding = 'utf-8'
#         soup = BeautifulSoup(r.text,'lxml')
#         df = pd.DataFrame(columns=['title'])
#         df2 = pd.DataFrame(columns=['content'])
#         for title in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-title'):
#             df = df.append({'title':title.text[:-1]},ignore_index=True)
#         for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
#             df2 = df2.append({'content':content.text},ignore_index=True)
#         result_m_f = pd.concat([df,df2],axis=1).loc[lambda result_m_f: result_m_f['title'] == '运作状态', :]
#         if str(result_m_f.content.iloc[0]) == '正在运作':
#             num3 += 1
#     df_m_row.loc[len(df_m_row),'content'] = num3
#     #result_m = result_m.append({'title':'存续产品数量', 'content':num3},ignore_index=True)

#     # 累计发行产品数量
#     num1=num2=0
#     try:
#         num2 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施后成立的基金', :].iloc[0,1].split('月报'))-1
#         num1 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施前成立的基金', :].iloc[0,1].split('月报'))-1
#     except:
#         pass
#     finally:
#         #result_m = result_m.append({'title':'累计发行产品数量', 'content':num1+num2},ignore_index=True)
#         df_m_row.loc[len(df_m_row),'content'] = num1+num2

#     row = df_m_row.transpose()
# ###
    return row

In [16]:
# Append df_m
for url_m in tqdm(df['网址']):
    row = get_result_m(url_m)
    df_m = df_m.append(pd.Series(row.values,index=index_m),ignore_index=True)
    #time.sleep(0.05)
    
# Save df_m
df_m.to_excel('funds_info.xlsx',encoding='gb18030')

  0%|          | 58/23580 [08:10<55:13:00,  8.45s/it]

ValueError: Wrong number of items passed 35, placement implies 33

In [15]:
df_m.tail(20)

,机构诚信信息,基金管理人全称(中文),基金管理人全称(英文),登记编号,组织机构代码,登记时间,成立时间,注册地址,办公地址,注册资本(万元)(人民币),...,资格取得方式,法定代表人/执行事务合伙人(委派代表)工作履历,高管情况,暂行办法实施前成立的基金,暂行办法实施后成立的基金,机构信息最后更新时间,特别提示信息,查询网址/二维码,存续产品数量,累计发行产品数量
0,,平安道远投资管理（上海）有限公司,Ping An DaoYuan Investment Management (Shangha...,P1000182,57075680-3,2014-03-17,2011-03-15,上海市浦东新区浦东新区张杨路707号生命人寿大厦39楼03室,上海市浦东新区浦东新区陆家嘴环路1333号平安金融大厦11楼,"20,000",...,资格认定,时间\n任职单位\n职务\n\n\n\n\n2015.09 -\r\n ...,高管姓名\n职务\n是否具有基金从业资格\n\n\n\n\n杨晓华\n法定代表人 总经理 ...,平安资产MOM1号资产管理产品\n当月月报：应披露1条，未披露0条；季报：应披露0条，未披露...,平安道远投资管理（上海）有限公司*稳盈8号私募投资基金 \n当月月报：应披露0条，未披露0条...,2018-04-19,,http://gs.amac.org.cn/amac-infodisc/res/pof/ma...,54,72
1,,上海重阳投资管理股份有限公司,"Shanghai Chongyang Investment Management Co., Ltd",P1000265,69155453-8,2014-03-17,2009-06-26,上海市浦东新区浦东新区世纪大道8号国金中心B座51楼5103-5116室,上海市浦东新区浦东新区世纪大道8号国金中心二期51楼,"23,500",...,资格认定,时间\n任职单位\n职务\n\n\n\n\n2009.06 -\r\n ...,高管姓名\n职务\n是否具有基金从业资格\n\n\n\n\n裘国根\n法定代表人 董事长 ...,华润信托·重阳3期证券投资集合资金信托计划\n当月月报：应披露1条，未披露0条；季报：应披露...,重阳灵活策略钜享价值2期私募基金 \n当月月报：应披露1条，未披露0条；季报：应披露1条，未...,2018-04-27,,http://gs.amac.org.cn/amac-infodisc/res/pof/ma...,61,83
2,,上海景林资产管理有限公司,Shanghai Greenwoods Asset Management Co. Ltd.,P1000267,59812186-9,2014-03-17,2012-06-06,上海市浦东新区浦东新区海徐路,上海市浦东新区浦东新区芳甸路1155号嘉里城办公楼27楼,"3,000",...,通过考试,时间\n任职单位\n职务\n\n\n\n\n2014.07 -\r\n ...,高管姓名\n职务\n是否具有基金从业资格\n\n\n\n\n蒋锦志\n董事长\n\r\n ...,得利宝.至尊14号\n当月月报：应披露1条，未披露0条；季报：应披露0条，未披露0条；半年报...,华润信托·景林鑫悦香江一期港股专户单一资金信托 \n当月月报：应披露1条，未披露0条；季报：...,2018-01-25,,http://gs.amac.org.cn/amac-infodisc/res/pof/ma...,143,159


## Append df_m的性能优化

V1.0 原始功能实现，性能优化 2018-5-11
[04:57<88:37:39, 13.54s/it]

V1.1 删除函数get_result_m中的df1冗余项 **Debugging Now**

V1.2 函数get_result_m中添加并加快`存续产品数量`查询的速度

v1.3 函数get_result_m中使用numpy代替pandas

V2.0 完善Result_m中复杂项目信息的可读性（人员信息等）

V3.0 使用Scrapy的多线程爬虫


In [ ]:
# Developing Phrase
# V1.1 
%%time

headers = {'User-Agent': UserAgent().random}
r = requests.get(url_m,headers)
r.encoding = 'utf-8'
soup = BeautifulSoup(r.text,'lxml')

df_m_row = pd.DataFrame(columns=['content'])
for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
    df_m_row = df_m_row.append({'content':content.text.strip(' &nbsp\r\n')},ignore_index=True)
df_m_row = df_m_row[df_m_row['content'] != '']
df_m_row = df_m_row.reset_index(drop=True)
df_m_row.loc[len(df_m_row),'content'] = url_m
df_m_row.content[0] = df_m_row.content[0].split('\n')[1]

# 存续产品数量
## 程序改进：合并dataframe速度提升？
num3 = 0
for fund in soup.find_all('a',href=True,class_=False,onclick=False):
    url = str('http://gs.amac.org.cn/amac-infodisc/res/pof/fund/'+fund['href'].split('/')[2])
    headers = {'User-Agent': UserAgent().random}
    r = requests.get(url,headers)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text,'lxml')
    df = pd.DataFrame(columns=['title'])
    df2 = pd.DataFrame(columns=['content'])
    for title in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-title'):
        df = df.append({'title':title.text[:-1]},ignore_index=True)
    for content in soup.find('table',class_='table table-center table-info').find_all('td',class_='td-content'):
        df2 = df2.append({'content':content.text},ignore_index=True)
    result_m_f = pd.concat([df,df2],axis=1).loc[lambda result_m_f: result_m_f['title'] == '运作状态', :]
    if str(result_m_f.content.iloc[0]) == '正在运作':
        num3 += 1
df_m_row.loc[len(df_m_row),'content'] = num3
#result_m = result_m.append({'title':'存续产品数量', 'content':num3},ignore_index=True)

# 累计发行产品数量
num1=num2=0
try:
    num2 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施后成立的基金', :].iloc[0,1].split('月报'))-1
    num1 = len(result_m.loc[lambda result_m: result_m['title'] == '暂行办法实施前成立的基金', :].iloc[0,1].split('月报'))-1
except:
    pass
finally:
    #result_m = result_m.append({'title':'累计发行产品数量', 'content':num1+num2},ignore_index=True)
    df_m_row.loc[len(df_m_row),'content'] = num1+num2
    
row = df_m_row.transpose()

In [ ]:
df_m_row